# generate working files
files are feature wiese or station wise with columns Date and Stations/features with header

In [1]:
import os
import sys
from typing import Optional
import polars as pl
import yaml


def load_data(data_dir: str, data_file: str) -> pl.DataFrame:
    ''' Load and preprocess data from the specified directory.
    '''
    df = pl.read_csv(os.path.join(data_dir, data_file), try_parse_dates=True)
    return df


def generate_working_files(dir: str = "../data/04_working", df: Optional[pl.DataFrame] = None, partition_by: str = "Feature") -> None:
    ''' Generate working files partitioned by Feature or Station.
    '''
    if df is None:
        print("DataFrame is None, please provide a valid DataFrame.")
        return

    stations = df["Station"].unique().to_list()
    features = df["Feature"].unique().to_list()
    DIR = dir
    if not os.path.exists(DIR):
        os.makedirs(DIR)


    file_list = []
    file_list_missing = []
    df = df.with_columns(pl.col("Value").interpolate(),)

    if partition_by == "Feature":
        for feature in features:
            ts_feature_file = os.path.join(DIR, f"ts_feature_{feature}.csv")
            ts_missing_feature_file = os.path.join(DIR, f"ts_feature_{feature}_missing.csv")
            df_feature = df.filter(pl.col("Feature") == feature)
        # Further processing...
            df_ts = df_feature.select([pl.col("Date"), pl.col("Station"), pl.col("Value")]).pivot(index="Date", on="Station", values="Value")
            df_ts_m = df_feature.select([pl.col("Date"), pl.col("Station"), pl.col("Value_missing")]).pivot(index="Date", on="Station", values="Value_missing")
            df_ts.write_csv(ts_feature_file, separator=' ', null_value='nan')
            df_ts_m.write_csv(ts_missing_feature_file, separator=' ', null_value='nan')
            file_list.append(ts_feature_file)
            file_list_missing.append(ts_missing_feature_file)

    elif partition_by == "Station":
        for station in stations:
            ts_station_file = os.path.join(DIR, f"ts_station_{station}.csv")
            ts_missing_station_file = os.path.join(DIR, f"ts_station_{station}_missing.csv")
            df_station = df.filter(pl.col("Station") == station)
        # Further processing...
            df_ts = df_station.select([pl.col("Date"), pl.col("Feature"), pl.col("Value")]).pivot(index="Date", on="Feature", values="Value")
            df_ts_m = df_station.select([pl.col("Date"), pl.col("Feature"), pl.col("Value_missing")]).pivot(index="Date", on="Feature", values="Value_missing")
            df_ts.write_csv(ts_station_file, separator=' ', null_value='nan')
            df_ts_m.write_csv(ts_missing_station_file, separator=' ', null_value='nan')
            file_list.append(ts_station_file)
            file_list_missing.append(ts_missing_station_file)

    #save file lists
    with open(os.path.join(DIR, "file_list.yaml"), 'w') as file:
        yaml.dump(file_list, file)
    with open(os.path.join(DIR, "file_list_missing.yaml"), 'w') as file:
        yaml.dump(file_list_missing, file)

def main():
    if len(sys.argv) != 4:
        print("Usage: python prepare_working_files.py <data_dir> <data_file> <data_dir_output>")
        sys.exit(1)

    data_dir = sys.argv[1]
    data_file = sys.argv[2]
    data_dir_output = sys.argv[3]

    df = load_data(data_dir, data_file)
    generate_working_files(dir=data_dir_output, df=df, partition_by="Feature")

if __name__ == "__main__":
    main()

Usage: python prepare_working_files.py <data_dir> <data_file> <data_dir_output>


SystemExit: 1

/home/thurin/Documents/timeseriesimputation/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# generate data dicts

In [6]:
import os
import sys
from typing import Optional
import joblib
import polars as pl
import pandas as pd
import yaml
from sklearn.preprocessing import MinMaxScaler

def load_data(data_dir: str, data_file: str) -> pl.DataFrame:
    ''' Load and preprocess data from the specified directory.
    '''
    df = pl.read_csv(os.path.join(data_dir, data_file), try_parse_dates=True)
    return df


def generate_working_files(dir: str = "../data/04_working", df: Optional[pl.DataFrame] = None, 
                           partition_by: str = "Feature", scaling: bool = False, scaler_dir: str = "../data/03_scalers", resampling: bool = False) -> None:
    ''' Generate working files partitioned by Feature or Station.
    '''
    if df is None:
        print("DataFrame is None, please provide a valid DataFrame.")
        return

    DIR = dir
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    df_pandas = df.to_pandas()
    df_pandas_wide = df_pandas.pivot(index="Date", columns=["Station", "Feature"], values="Value")
    df_pandas_wide_missing = df_pandas.pivot(index="Date", columns=["Station", "Feature"], values="Value_missing")
    df_pandas_wide_mask = df_pandas.pivot(index="Date", columns=["Station", "Feature"], values="missing")

    if scaling:
        scaler = MinMaxScaler()
        scaler.fit(df_pandas_wide.loc[:, :])
        df_pandas_wide_missing.loc[:, :] = scaler.transform(df_pandas_wide_missing.loc[:, :])
        if not os.path.exists(scaler_dir):
            os.makedirs(scaler_dir)
        scaler_file = os.path.join(scaler_dir, "scaler_global.pkl")
        joblib.dump(scaler, scaler_file)

    def add_features(df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(pl.col("Date").dt.year().alias("Year"),
                              pl.col("Date").dt.month().alias("Month"),
                              pl.col("Date").dt.day().alias("Day"),
                              pl.col("Date").dt.hour().alias("Hour") if not resampling else pl.lit(0).alias("Hour"))
    
    def save_data(data_dict: dict, filename: str) -> None:
        with open(os.path.join(DIR, filename), "wb") as f:
            joblib.dump(data_dict, f)
            #
    def scale_datetime_features(df: pd.DataFrame) -> pd.DataFrame:
        datetime_scaler = MinMaxScaler()
        datetime_features = df[["Year", "Month", "Day", "Hour"]]
        df[["Year", "Month", "Day", "Hour"]] = datetime_scaler.fit_transform(datetime_features)
        return df


    if partition_by == "Feature":
        df_long = pl.from_pandas(df_pandas_wide.melt(ignore_index=False),include_index=True)
        df_long_missing = pl.from_pandas(df_pandas_wide_missing.melt(ignore_index=False),include_index=True)
        df_long_mask = pl.from_pandas(df_pandas_wide_mask.melt(ignore_index=False),include_index=True)

        data = df_long.partition_by("Feature", as_dict=True)
        data_missing = df_long_missing.partition_by("Feature", as_dict=True)
        data_mask = df_long_mask.partition_by("Feature", as_dict=True)

        data_pandas, data_missing_pandas, data_mask_pandas = {}, {}, {}

        for key in data_missing.keys():
            data_pandas[key[0]] = data[key].pivot(index="Date", on="Station", values="value").to_pandas().set_index('Date')
            data_missing_pandas[key[0]] = scale_datetime_features(add_features(data_missing[key].pivot(index="Date", on="Station", values="value")).to_pandas().set_index('Date'))
            data_mask_pandas[key[0]] = data_mask[key].pivot(index="Date", on="Station", values="value").to_pandas().set_index('Date')
        
        save_data(data_pandas, "ts_feature_gt.pkl")
        save_data(data_missing_pandas, "ts_feature_missing.pkl")
        save_data(data_mask_pandas, "ts_feature_mask.pkl")

    elif partition_by == "Station":
        df_long = pl.from_pandas(df_pandas_wide.melt(ignore_index=False),include_index=True)
        df_long_missing = pl.from_pandas(df_pandas_wide_missing.melt(ignore_index=False),include_index=True)
        df_long_mask = pl.from_pandas(df_pandas_wide_mask.melt(ignore_index=False),include_index=True)

        data = df_long.partition_by("Station", as_dict=True)
        data_missing = df_long_missing.partition_by("Station", as_dict=True)
        data_mask = df_long_mask.partition_by("Station", as_dict=True)

        data_pandas, data_missing_pandas, data_mask_pandas = {}, {}, {}

        for key in data_missing.keys():
            data_pandas[key[0]] = data[key].pivot(index="Date", on="Feature", values="value").to_pandas().set_index('Date')
            data_missing_pandas[key[0]] = scale_datetime_features(add_features(data_missing[key].pivot(index="Date", on="Feature", values="value")).to_pandas().set_index('Date'))
            data_mask_pandas[key[0]] = data_mask[key].pivot(index="Date", on="Feature", values="value").to_pandas().set_index('Date')

        save_data(data_pandas, "ts_station_gt.pkl")
        save_data(data_missing_pandas, "ts_station_missing.pkl")
        save_data(data_mask_pandas, "ts_station_mask.pkl")

    elif partition_by == "Wide":
        df_pandas_wide_missing["Year"] = df_pandas_wide_missing.index.year
        df_pandas_wide_missing["Month"] = df_pandas_wide_missing.index.month
        df_pandas_wide_missing["Day"] = df_pandas_wide_missing.index.day
        df_pandas_wide_missing["Hour"] = df_pandas_wide_missing.index.hour if not resampling else 0
        df_pandas_wide_missing = scale_datetime_features(df_pandas_wide_missing)
        
        data_pandas, data_missing_pandas, data_mask_pandas = {}, {}, {}

        data_pandas["Wide"] = df_pandas_wide
        data_missing_pandas["Wide"] = df_pandas_wide_missing
        data_mask_pandas["Wide"] = df_pandas_wide_mask

        save_data(data_pandas, "ts_wide_gt.pkl")
        save_data(data_missing_pandas, "ts_wide_missing.pkl")
        save_data(data_mask_pandas, "ts_wide_mask.pkl")


def main():
    if len(sys.argv) != 5:
        print("Usage: python prepare_working_files.py <data_dir> <data_file> <data_dir_output>")
        sys.exit(1)

    data_dir = sys.argv[1]
    data_file = sys.argv[2]
    data_dir_output = sys.argv[3]
    scaler_dir = sys.argv[4]

    with open("params.yaml", 'r') as file:
        params = yaml.safe_load(file)
    partition_by = params["prepare_working_files"]["partition_by"]
    scaling = params["prepare_working_files"]["scaling"]
    resampling = params["preprocessing"]["resample"]

    df = load_data(data_dir, data_file)
    generate_working_files(dir=data_dir_output, df=df, partition_by=partition_by, scaling=scaling, scaler_dir=scaler_dir, resampling=resampling)

if __name__ == "__main__":
    main()

Usage: python prepare_working_files.py <data_dir> <data_file> <data_dir_output>


SystemExit: 1

/home/thurin/Documents/timeseriesimputation/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
df = load_data("../data/03_prepared/", "prepared_ref.csv")

In [42]:
generate_working_files(dir="../data/04_working/", df=df, partition_by="Station", scaling=True, scaler_dir="../data/03_scalers/", resampling=True)

In [ ]:
import os
import yaml
import joblib
import pandas as pd
import polars as pl


def get_working_files(dir: str = "../data/04_working/") -> tuple[list[str], list[str]]:
    """Get list of working files and files with missing values from YAML files.

    Args:
        dir (str): Directory where the YAML files are located. Defaults to "../data/04_working_files/".

    Returns:
        tuple: A tuple containing two lists:
            - List of working file paths.
            - List of file paths with missing values.
    """
    datafiles = []
    datafiles_missing = []
    files = os.listdir(dir)
    for file in files:
        if "gt" in file:
            datafiles.append(os.path.join(dir, file))
        if "missing" in file:
            datafiles_missing.append(os.path.join(dir, file))

    return datafiles, datafiles_missing

def load_working_files(data_files: list[str], missing_data_files: list[str]) -> tuple[dict, dict, str]:
    """Load working files and files with missing values.

    Args:
        data_files (str): Path to the working file.
        missing_data_files (str): Path to the file with missing values.
    Returns:
        tuple: A tuple containing two dictionaries:
            - Dictionary of working data dataframes.
            - Dictionary of missing data dataframes.
    """
    working_data = joblib.load(data_files[0])
    missing_data = joblib.load(missing_data_files[0])
    prep_method = data_files[0].split("/")[-1].split("_")[1]
    return working_data, missing_data, prep_method

def save_imputed_files(imputed_data: dict, output_dir: str = "../data/05_imputed_data/", 
                       imputation_method: str = "ffill", prep_method: str = "station"):
    """Save imputed dataframes to CSV files.

    Args:
        imputed_data (dict): Dictionary of imputed dataframes.
        output_dir (str): Directory to save the imputed files. Defaults to "../data/05_imputed_data/".
        imputation_method (str): Imputation method used. Defaults to "ffil".
        prep_method (str): Preparation method used. Defaults to "station".
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_dir = os.path.join(output_dir, imputation_method)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    temp_df = pd.DataFrame()

    time_features = ["Year", "Month", "Day", "Hour"]

    for key, df in imputed_data.items():
        if any(feature in df.columns for feature in time_features):
            df = df.drop(columns=[feature for feature in time_features if feature in df.columns])
        if prep_method == "wide":
            df = df.melt(ignore_index=False, var_name=["Station", "Feature"], value_name="Value")
        elif prep_method == "feature":
            df = df.melt(ignore_index=False, var_name="Station", value_name="Value")
            df["Feature"] = key
        elif prep_method == "station":
            df = df.melt(ignore_index=False, var_name="Feature", value_name="Value")
            df["Station"] = key
        else:
            raise ValueError(f"Unknown prep_method: {prep_method}")

        # Collect all processed DataFrames
        temp_df = pd.concat([temp_df, df], axis=0)

    # Save once, after all processing
    output_path = os.path.join(output_dir, f"ts_{prep_method}_imputed.csv")
    temp_df.to_csv(output_path, index=True, sep=',')
    print(f"✅ Imputed data saved to {output_path}")
        # file_name = f"ts_{prep_method}_{key}.csv"
        # file_name = file_name.replace("missing", "imputed")
        # output_path = os.path.join(output_dir, file_name)
        # df.to_csv(output_path, index=True, sep=',')
        # print(f"Imputed data saved to {output_path}")

In [4]:
os.listdir("../data/04_working/")

['ts_wide_gt.pkl', 'ts_wide_missing.pkl', 'ts_wide_mask.pkl']

In [9]:
generate_working_files(dir="../data/04_working/", df=df, partition_by="Wide", scaling=True, scaler_dir="../data/03_scalers/", resampling=True)

In [16]:
data_files, missing_data_files = get_working_files(dir="../data/04_working/")
working_data, missing_data, prep_method = load_working_files(data_files, missing_data_files)

TypeError: get_working_files() missing 1 required positional argument: 'mask'

In [11]:
save_imputed_files(working_data, output_dir="../data/05_imputed_data/", imputation_method="ground_truth", prep_method=prep_method)

✅ Imputed data saved to ../data/05_imputed_data/ground_truth/ts_wide_imputed.csv


In [44]:
missing_data["Wide"]

Station,5904_WIEN-HOHE,5925_WIEN-INNERE,3202_LINZ-STADT,4305_ZWERNDORF,12504_BISCHOFSHOFEN,5316_AMSTETTEN,11505_REUTTE,7604_WR.NEUSTADT/FLUGPLATZ,5972_GROSS-ENZERSDORF,5904_WIEN-HOHE,...,4305_ZWERNDORF,12504_BISCHOFSHOFEN,5316_AMSTETTEN,11505_REUTTE,7604_WR.NEUSTADT/FLUGPLATZ,5972_GROSS-ENZERSDORF,Year,Month,Day,Hour
Feature,Wind_direction,Wind_direction,Wind_direction,Wind_direction,Wind_direction,Wind_direction,Wind_direction,Wind_direction,Wind_direction,Relative_humidity,...,Wind_speed,Wind_speed,Wind_speed,Wind_speed,Wind_speed,Wind_speed,,,,
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-05,0.771743,0.755005,0.796089,0.779746,NaN,0.864257,NaN,NaN,NaN,0.448761,...,0.375448,NaN,0.335835,NaN,NaN,NaN,0.0,0.636364,0.133333,0.0
2008-08-06,0.493714,0.390281,0.489292,0.450037,NaN,0.315967,NaN,NaN,NaN,0.369056,...,0.039458,NaN,0.102564,NaN,NaN,NaN,0.0,0.636364,0.166667,0.0
2008-08-07,0.294621,0.250066,0.533675,0.363548,NaN,0.119617,NaN,NaN,NaN,0.401206,...,0.312077,NaN,0.359600,NaN,NaN,NaN,0.0,0.636364,0.200000,0.0
2008-08-08,0.727155,0.680122,0.774209,0.682140,NaN,0.798334,NaN,NaN,NaN,0.553248,...,0.304105,NaN,0.257036,NaN,NaN,NaN,0.0,0.636364,0.233333,0.0
2008-08-09,0.856254,0.812586,0.849938,0.853653,NaN,0.885876,NaN,NaN,NaN,0.527127,...,0.404145,NaN,0.308318,NaN,NaN,NaN,0.0,0.636364,0.266667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-31,0.725340,NaN,0.532899,NaN,0.536230,0.455254,NaN,NaN,0.880619,0.239786,...,NaN,0.298861,0.119450,NaN,NaN,0.387157,1.0,0.545455,1.000000,0.0
2018-08-01,0.531432,NaN,0.496276,NaN,0.520143,0.357611,NaN,NaN,0.668342,0.383791,...,NaN,0.322524,0.109443,NaN,NaN,0.173569,1.0,0.636364,0.000000,0.0


In [8]:
df_wide = df.pivot(index="Date", on=["Station", "Feature"], values="Value")

In [147]:
import pickle
df_pandas = df.to_pandas()
df_wide_pandas = df_pandas.pivot(index="Date", columns=["Station", "Feature"], values="Value")
polars_dict = pl.from_pandas(df_wide_pandas,include_index=True).partition_by("Station", as_dict=True)


ColumnNotFoundError: "Station" not found

In [149]:
print(df_wide_pandas.head())

Station    5904_WIEN-HOHE 5925_WIEN-INNERE 3202_LINZ-STADT 4305_ZWERNDORF  \
Feature    Wind_direction   Wind_direction  Wind_direction Wind_direction   
Date                                                                        
2008-08-05     270.208333       267.458333      241.208333     272.500000   
2008-08-06     180.833333       157.666667      158.833333     160.041667   
2008-08-07     116.833333       115.458333      170.750000     130.541667   
2008-08-08     255.875000       244.916667      235.333333     239.208333   
2008-08-09     297.375000       284.791667      255.666667     297.708333   

Station    12504_BISCHOFSHOFEN 5316_AMSTETTEN   11505_REUTTE  \
Feature         Wind_direction Wind_direction Wind_direction   
Date                                                           
2008-08-05          171.541667     257.375000     263.875000   
2008-08-06          214.291667     128.458333     244.458333   
2008-08-07          152.708333      82.291667     242.208333   

In [119]:
for key in polars_dict.keys():
    print(key[0])

5904_WIEN-HOHE
5925_WIEN-INNERE
3202_LINZ-STADT
4305_ZWERNDORF
12504_BISCHOFSHOFEN
20212_KLAGENFURT-FLUGHAFEN
5316_AMSTETTEN
11505_REUTTE
5972_GROSS-ENZERSDORF


In [117]:
polars_dict[('5904_WIEN-HOHE',)].pivot(index="Date", on="Feature", values="value").to_pandas().set_index('Date')

,Wind_direction,Relative_humidity,Air_pressure,Air_temperature,Wind_speed
Date,,,,,
2008-08-05 00:00:00,270.0,82.0,988.9,292.65,4.3
2008-08-05 01:00:00,263.0,85.0,988.8,291.75,4.3
2008-08-05 02:00:00,263.0,79.0,988.6,291.55,4.3
2008-08-05 03:00:00,271.0,76.0,988.5,291.55,4.9
2008-08-05 04:00:00,268.0,79.0,988.7,291.45,4.4
...,...,...,...,...,...
2018-08-04 19:00:00,157.0,43.0,991.5,301.65,2.4
2018-08-04 20:00:00,228.0,52.0,991.6,299.45,1.5
2018-08-04 21:00:00,236.0,57.0,991.5,298.55,2.0


In [41]:
data_dict["ts_wide"]

,Station,Feature,value
Date,,,
2008-08-05 00:00:00,5904_WIEN-HOHE,Wind_direction,270.0
2008-08-05 01:00:00,5904_WIEN-HOHE,Wind_direction,263.0
2008-08-05 02:00:00,5904_WIEN-HOHE,Wind_direction,263.0
2008-08-05 03:00:00,5904_WIEN-HOHE,Wind_direction,271.0
2008-08-05 04:00:00,5904_WIEN-HOHE,Wind_direction,268.0
...,...,...,...
2018-08-04 19:00:00,5972_GROSS-ENZERSDORF,Wind_speed,1.3
2018-08-04 20:00:00,5972_GROSS-ENZERSDORF,Wind_speed,1.8
2018-08-04 21:00:00,5972_GROSS-ENZERSDORF,Wind_speed,2.1


In [42]:
# load from pickle
with open("../data/data_dict.pkl", "rb") as f:
    data_dict_loaded = pickle.load(f)
data_dict_loaded["ts_wide"]

,Station,Feature,value
Date,,,
2008-08-05 00:00:00,5904_WIEN-HOHE,Wind_direction,270.0
2008-08-05 01:00:00,5904_WIEN-HOHE,Wind_direction,263.0
2008-08-05 02:00:00,5904_WIEN-HOHE,Wind_direction,263.0
2008-08-05 03:00:00,5904_WIEN-HOHE,Wind_direction,271.0
2008-08-05 04:00:00,5904_WIEN-HOHE,Wind_direction,268.0
...,...,...,...
2018-08-04 19:00:00,5972_GROSS-ENZERSDORF,Wind_speed,1.3
2018-08-04 20:00:00,5972_GROSS-ENZERSDORF,Wind_speed,1.8
2018-08-04 21:00:00,5972_GROSS-ENZERSDORF,Wind_speed,2.1


In [20]:
import polars.selectors as cs
df_wide#.unpivot(on=cs.numeric(), value_name="Value")#.unnest("variable")

Date,"{""5904_WIEN-HOHE"",""Wind_direction""}","{""5925_WIEN-INNERE"",""Wind_direction""}","{""3202_LINZ-STADT"",""Wind_direction""}","{""4305_ZWERNDORF"",""Wind_direction""}","{""12504_BISCHOFSHOFEN"",""Wind_direction""}","{""20212_KLAGENFURT-FLUGHAFEN"",""Wind_direction""}","{""5316_AMSTETTEN"",""Wind_direction""}","{""11505_REUTTE"",""Wind_direction""}","{""5972_GROSS-ENZERSDORF"",""Wind_direction""}","{""5904_WIEN-HOHE"",""Relative_humidity""}","{""5925_WIEN-INNERE"",""Relative_humidity""}","{""3202_LINZ-STADT"",""Relative_humidity""}","{""4305_ZWERNDORF"",""Relative_humidity""}","{""12504_BISCHOFSHOFEN"",""Relative_humidity""}","{""20212_KLAGENFURT-FLUGHAFEN"",""Relative_humidity""}","{""5316_AMSTETTEN"",""Relative_humidity""}","{""11505_REUTTE"",""Relative_humidity""}","{""5972_GROSS-ENZERSDORF"",""Relative_humidity""}","{""5904_WIEN-HOHE"",""Air_pressure""}","{""5925_WIEN-INNERE"",""Air_pressure""}","{""3202_LINZ-STADT"",""Air_pressure""}","{""4305_ZWERNDORF"",""Air_pressure""}","{""12504_BISCHOFSHOFEN"",""Air_pressure""}","{""20212_KLAGENFURT-FLUGHAFEN"",""Air_pressure""}","{""5316_AMSTETTEN"",""Air_pressure""}","{""11505_REUTTE"",""Air_pressure""}","{""5972_GROSS-ENZERSDORF"",""Air_pressure""}","{""5904_WIEN-HOHE"",""Air_temperature""}","{""5925_WIEN-INNERE"",""Air_temperature""}","{""3202_LINZ-STADT"",""Air_temperature""}","{""4305_ZWERNDORF"",""Air_temperature""}","{""12504_BISCHOFSHOFEN"",""Air_temperature""}","{""20212_KLAGENFURT-FLUGHAFEN"",""Air_temperature""}","{""5316_AMSTETTEN"",""Air_temperature""}","{""11505_REUTTE"",""Air_temperature""}","{""5972_GROSS-ENZERSDORF"",""Air_temperature""}","{""5904_WIEN-HOHE"",""Wind_speed""}","{""5925_WIEN-INNERE"",""Wind_speed""}","{""3202_LINZ-STADT"",""Wind_speed""}","{""4305_ZWERNDORF"",""Wind_speed""}","{""12504_BISCHOFSHOFEN"",""Wind_speed""}","{""20212_KLAGENFURT-FLUGHAFEN"",""Wind_speed""}","{""5316_AMSTETTEN"",""Wind_speed""}","{""11505_REUTTE"",""Wind_speed""}","{""5972_GROSS-ENZERSDORF"",""Wind_speed""}"
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2008-08-05 00:00:00,270.0,266.0,216.0,293.0,134.0,286.0,239.0,184.0,263.0,82.0,73.0,90.0,93.0,95.0,96.0,91.0,88.0,75.0,988.9,992.6,984.8,994.8,952.7,962.3,982.3,919.9,995.4,292.65,295.45,290.55,293.15,289.95,291.15,290.65,288.95,294.15,4.3,3.9,0.8,0.5,0.3,0.9,2.0,1.0,2.9
2008-08-05 01:00:00,263.0,246.0,243.0,298.0,131.0,287.0,272.0,220.0,269.0,85.0,80.0,89.0,87.0,95.0,96.0,88.0,89.0,75.0,988.8,992.4,984.6,994.7,952.3,962.5,982.1,919.8,995.4,291.75,293.75,290.15,291.95,289.35,291.25,290.85,288.65,293.55,4.3,4.3,0.7,3.5,0.6,0.6,1.4,0.7,3.3
2008-08-05 02:00:00,263.0,246.0,241.0,255.0,152.0,293.0,255.0,360.0,263.0,79.0,76.0,87.0,86.0,95.0,96.0,92.0,88.0,70.0,988.6,992.4,984.6,994.6,952.1,962.3,982.1,920.3,995.1,291.55,293.45,289.55,291.75,289.35,290.85,290.45,288.75,293.35,4.3,4.4,1.0,2.5,0.6,1.4,1.3,1.4,3.2
2008-08-05 03:00:00,271.0,261.0,230.0,276.0,139.0,323.0,249.0,351.0,272.0,76.0,77.0,88.0,86.0,95.0,96.0,89.0,93.0,70.0,988.5,992.3,984.3,994.6,952.0,961.9,982.0,920.8,995.2,291.55,293.05,289.25,291.35,289.25,290.65,290.35,288.45,292.95,4.9,3.5,1.1,2.6,1.2,0.4,1.8,2.1,3.4
2008-08-05 04:00:00,268.0,257.0,217.0,281.0,127.0,324.0,265.0,353.0,270.0,79.0,78.0,90.0,85.0,94.0,96.0,95.0,92.0,75.0,988.7,992.4,984.7,994.9,952.3,962.2,982.5,921.2,995.4,291.45,292.95,289.15,291.15,288.75,290.65,289.65,287.55,292.45,4.4,3.4,1.1,2.2,0.7,1.0,1.3,2.4,3.2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-08-04 19:00:00,157.0,149.0,311.0,151.0,27.0,69.0,161.0,355.0,148.0,43.0,36.0,50.0,56.0,45.0,48.0,66.0,67.0,53.0,991.5,995.1,984.8,998.7,954.1,964.6,984.9,923.4,998.0,301.65,303.65,301.05,299.25,299.85,300.25,297.55,295.15,300.15,2.4,2.6,0.5,1.9,1.5,2.2,0.1,0.4,1.3
2018-08-04 20:00:00,228.0,160.0,297.0,149.0,142.0,76.0,251.0,126.0,158.0,52.0,40.0,

In [ ]:
from pypots.imputation import timem

# testing of pypots

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar, calc_missing_rate
from benchpots.datasets import preprocess_physionet2012
data = preprocess_physionet2012(subset='set-a',rate=0.1) # Our ecosystem libs will automatically download and extract it
train_X, val_X, test_X = data["train_X"], data["val_X"], data["test_X"]
print(train_X.shape)  # (n_samples, n_steps, n_features)
print(val_X.shape)  # samples (n_samples) in train set and val set are different, but they have the same sequence len (n_steps) and feature dim (n_features)
print(f"We have {calc_missing_rate(train_X):.1%} values missing in train_X")  
train_set = {"X": train_X}  # in training set, simply put the incomplete time series into it
val_set = {
    "X": val_X,
    "X_ori": data["val_X_ori"],  # in validation set, we need ground truth for evaluation and picking the best model checkpoint
}
test_set = {"X": test_X}  # in test set, only give the testing incomplete time series for model to impute
test_X_ori = data["test_X_ori"]  # test_X_ori bears ground truth for evaluation
indicating_mask = np.isnan(test_X) ^ np.isnan(test_X_ori)  # mask indicates the values that are missing in X but not in X_ori, i.e. where the gt values are 

from pypots.imputation import SAITS  # import the model you want to use
from pypots.nn.functional import calc_mae
saits = SAITS(n_steps=train_X.shape[1], n_features=train_X.shape[2], n_layers=2, d_model=256, n_heads=4, d_k=64, d_v=64, d_ffn=128, dropout=0.1, epochs=150)
saits.fit(train_set, val_set)  # train the model on the dataset
imputation = saits.impute(test_set)  # impute the originally-missing values and artificially-missing values
mae = calc_mae(imputation, np.nan_to_num(test_X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)
saits.save("save_it_here/saits_physionet2012.pypots")  # save the model for future use
saits.load("save_it_here/saits_physionet2012.pypots")  # reload the serialized model file for following imputation or training

2025-10-21 09:42:04 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-10-21 09:42:04 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-10-21 09:42:04 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-10-21 09:42:05 [INFO]: Loaded successfully!
2025-10-21 09:42:11 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-10-21 09:42:11 [INFO]: 22956 values masked out in the val set as ground truth, take 10.04% of the original observed values
2025-10-21 09:42:11 [INFO]: 28956 values masked out in the test set as ground truth, take 9.98% of the original observed values
2025-10-21 09:42:11 [INFO]: Total sample number: 3997
2025-10-21 09:42:11 [IN

(2557, 48, 37)
(640, 48, 37)
We have 79.7% values missing in train_X


2025-10-21 09:42:12.071201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-21 09:42:12.093856: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-21 09:42:12.093908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-21 09:42:12.120874: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-21 09:42:13.006814: W tensorflow/compiler/tf


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



2025-10-21 09:42:18 [INFO]: Epoch 001 - training loss (MAE): 0.9518, validation MSE: 0.4414
2025-10-21 09:42:19 [INFO]: Epoch 002 - training loss (MAE): 0.6556, validation MSE: 0.4022
2025-10-21 09:42:21 [INFO]: Epoch 003 - training loss (MAE): 0.5922, validation MSE: 0.3927
2025-10-21 09:42:22 [INFO]: Epoch 004 - training loss (MAE): 0.5567, validation MSE: 0.3821
2025-10-21 09:42:23 [INFO]: Epoch 005 - training loss (MAE): 0.5358, validation MSE: 0.3712
2025-10-21 09:42:25 [INFO]: Epoch 006 - training loss (MAE): 0.5148, validation MSE: 0.3565
2025-10-21 09:42:26 [INFO]: Epoch 007 - training loss (MAE): 0.4980, validation MSE: 0.3463
2025-10-21 09:42:27 [INFO]: Epoch 008 - training loss (MAE): 0.4832, validation MSE: 0.3291
2025-10-21 09:42:29 [INFO]: Epoch 009 - training loss (MAE): 0.4686, validation MSE: 0.3225
2025-10-21 09:42:30 [INFO]: Epoch 010 - training loss (MAE): 0.4571, validation MSE: 0.3128
2025-10-21 09:42:31 [INFO]: Epoch 011 - training loss (MAE): 0.4494, validation 

KeyboardInterrupt: 

In [4]:
mae

0.215173751566349

In [ ]:
test_X

array([[[        nan,         nan,         nan, ...,         nan,
         -0.30405806, -0.00604121],
        [        nan,         nan,         nan, ..., -0.76072109,
                 nan, -0.0072034 ],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
         -0.08401638, -0.01533871],
        [        nan,         nan,         nan, ...,         nan,
         -0.08401638,         nan],
        [        nan,         nan,         nan, ...,         nan,
         -0.08401638,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
          0.02993378,         nan],
        [        nan,         nan,         nan, ...,         nan,
          0.02993378,         nan],
        ...,
        [        nan,         nan,         nan, ...,  

In [6]:
train_X.shape

(2557, 48, 37)

# testing of impute gag
rewrite attempt of eval function of benchmark class

In [ ]:
from imputegap.recovery.manager import TimeSeries
from imputegap.tools import utils
from imputegap.recovery.benchmark import Benchmark
import datetime
import time
import math
import matplotlib.pyplot as plt
bench = Benchmark()
runs = 1

def eval(self, algorithms=["cdrec"], datasets=["eeg-alcohol"], ts_missing=["egg-alcohol"], patterns=["mcar"], x_axis=[0.05, 0.1, 0.2, 0.4, 0.6, 0.8], optimizers=["default_params"], metrics=["*"], save_dir="./imputegap_assets/benchmark", runs=1, normalizer="z_score", nbr_series=2500, nbr_vals=2500, dl_ratio=0.9, verbose=False):
        """
        Execute a comprehensive evaluation of imputation algorithms over multiple datasets and patterns.

        Parameters
        ----------
        algorithms : list of str
            List of imputation algorithms to test.
        datasets : list of str
            List of dataset names to evaluate.
        ts_missing : TimeSeries
            TimeSeries object containing the missing data.
        patterns : list of str
            List of contamination patterns to apply.
        x_axis : list of float
            List of missing rates for contamination.
        optimizers : list
            List of optimizers with their configurations.
        metrics : list of str
            List of metrics for evaluation.
        save_dir : str, optional
            Directory to save reports and plots (default is "./reports").
        runs : int, optional
            Number of executions with a view to averaging them
        normalizer : str, optional
            Normalizer to pre-process the data (default is "z_score").
        nbr_series : int, optional
            Number of series to take inside the dataset (default is 2500 (as the max values)).
        nbr_vals : int, optional
            Number of values to take inside the series (default is 2500 (as the max values)).
        dl_ratio : float, optional
            Training ratio for Deep Learning techniques (default is 0.8)
        verbose : bool, optional
            Whether to display the contamination information (default is False).

        Returns
        -------
        List
            List of all runs results, matrix with averaged scores and times for all levels

        Notes
        -----
        Runs contamination, imputation, and evaluation, then generates plots and a summary reports.
        """
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

        run_storage = []
        not_optimized = ["none"]
        mean_group = ["mean", "MeanImpute", "min", "MinImpute", "zero", "ZeroImpute", "MeanImputeBySeries", "meanimpute", "minimpute", "zeroimpute", "meanimputebyseries"]

        if not isinstance(algorithms, list):
            raise TypeError(f"'algorithms' must be a list, but got {type(algorithms).__name__}")
        if not isinstance(datasets, list):
            raise TypeError(f"'datasets' must be a list, but got {type(datasets).__name__}")
        if not isinstance(patterns, list):
            raise TypeError(f"'patterns' must be a list, but got {type(patterns).__name__}")
        if not isinstance(x_axis, list):
            raise TypeError(f"'x_axis' must be a list, but got {type(x_axis).__name__}")

        if "*" in metrics or "all" in metrics:
            metrics = utils.list_of_metrics()
        if "*" in metrics or "all" in algorithms:
            all_algs = utils.list_of_algorithms()
            algorithms = [item for item in all_algs if item.upper() != "MPIN"]

        directory_now = datetime.datetime.now()
        directory_time = directory_now.strftime("%y_%m_%d_%H_%M_%S")
        save_dir = save_dir + "/" + "bench_" + directory_time

        benchmark_time = time.time()
        for i_run in range(0, abs(runs)):
            for i, dataset in enumerate(datasets):
                runs_plots_scores = {}
                block_size_mcar = 10
                y_p_size = max(4, len(algorithms)*0.275)

                if verbose:
                    print("\n1. evaluation launch for", dataset, "\n")
                ts_test = TimeSeries()
                default_data = TimeSeries()

                header = False
                if dataset == "eeg-reading" or dataset == "eegreading":
                    header = True

                reshp = False
                default_data.load_series(data=utils.search_path(dataset), header=header, verbose=False)
                Mdef, Ndef = default_data.data.shape

                if Ndef > nbr_vals or Mdef > nbr_series:
                    reshp = True
                    print(f"\nThe dataset contains a large number of values {default_data.data.shape}, which may be too much for some algorithms to handle efficiently. Consider reducing the number of series or the volume of data.")
                default_data = None

                ts_test.load_series(data=utils.search_path(dataset), nbr_series=nbr_series, nbr_val=nbr_vals, header=header)
                _, N = ts_test.data.shape

                if reshp:
                    print(f"Benchmarking module has reduced the shape to {ts_test.data.shape}.\n")

                if N < 250:
                    print(f"The block size is too high for the number of values per series, reduce to 2\n")
                    block_size_mcar = 2

                if normalizer in utils.list_of_normalizers():
                    ts_test.normalize(verbose=verbose)

                for pattern in patterns:
                    if verbose:
                        print("\n2. contamination of", dataset, "with pattern", pattern, "\n")

                    for algorithm in algorithms:
                        has_been_optimized = False

                        if verbose:
                            print("\n3. algorithm evaluated", algorithm, "with", pattern, "\n")
                        else:
                            print(f"{algorithm} is tested with {pattern}, started at {time.strftime('%Y-%m-%d %H:%M:%S')}.")

                        incomp_data = TimeSeries()
                        incomp_data = incomp_data.load_series(data = ts_missing[i])

                        for optimizer in optimizers:
                            algo = utils.config_impute_algorithm(incomp_data=incomp_data, algorithm=algorithm, verbose=verbose)

                            if isinstance(optimizer, dict):
                                optimizer_gt = {"input_data": ts_test.data, **optimizer}
                                optimizer_value = optimizer.get('optimizer')  # or optimizer['optimizer']

                                if not has_been_optimized and algorithm not in mean_group and algorithm not in not_optimized:
                                    if verbose:
                                        print("\n5. AutoML to set the parameters", optimizer, "\n")
                                    i_opti = self._config_optimization(0.20, ts_test, pattern, algorithm, block_size_mcar)

                                    if utils.check_family("DeepLearning", algorithm):
                                        i_opti.impute(user_def=False, params=optimizer_gt, tr_ratio=0.80)
                                    else:
                                        i_opti.impute(user_def=False, params=optimizer_gt)

                                    utils.save_optimization(optimal_params=i_opti.parameters, algorithm=algorithm, dataset=dataset, optimizer="e")

                                    has_been_optimized = True
                                else:
                                    if verbose:
                                        print("\n5. AutoML already optimized...\n")

                                if algorithm not in mean_group and algorithm not in not_optimized:
                                    if i_opti.parameters is None:
                                        opti_params = utils.load_parameters(query="optimal", algorithm=algorithm, dataset=dataset, optimizer="e")
                                        if verbose:
                                            print("\n6. imputation", algorithm, "with optimal parameters from files", *opti_params)
                                    else:
                                        opti_params = i_opti.parameters
                                        if verbose:
                                            print("\n6. imputation", algorithm, "with optimal parameters from object", *opti_params)
                                else:
                                    if verbose:
                                        print("\n5. No AutoML launches without optimal params for", algorithm, "\n")
                                    opti_params = None
                            else:
                                if verbose:
                                    print("\n5. Default parameters have been set the parameters", optimizer, "for", algorithm, "\n")
                                optimizer_value = optimizer
                                opti_params = None

                            start_time_imputation = time.time()

                            if not self._benchmark_exception(dataset, algorithm, pattern, 0.2):
                                if utils.check_family("DeepLearning", algorithm) or utils.check_family("LLMs", algorithm):
                                    if 0.2 > round(1-dl_ratio, 2):
                                        algo.recov_data = incomp_data
                                    else:
                                        algo.impute(params=opti_params, tr_ratio=dl_ratio)
                                else:
                                    algo.impute(params=opti_params)
                            else:
                                algo.recov_data = incomp_data

                            end_time_imputation = time.time()

                            algo.score(input_data=ts_test.data, recov_data=algo.recov_data, verbose=False)

                            if "*" not in metrics and "all" not in metrics:
                                algo.metrics = {k: algo.metrics[k] for k in metrics if k in algo.metrics}

                            time_imputation = (end_time_imputation - start_time_imputation) * 1000
                            if time_imputation < 1:
                                time_imputation = 1
                            log_time_imputation = math.log10(time_imputation) if time_imputation > 0 else None

                            algo.metrics["RUNTIME"] = time_imputation
                            algo.metrics["RUNTIME_LOG"] = log_time_imputation

                            dataset_s = dataset
                            if "-" in dataset:
                                dataset_s = dataset.replace("-", "")

                            save_dir_plot = save_dir + "/" + dataset_s + "/" + pattern + "/recovery/"
                            cont_rate = int(x*100)
                            ts_test.plot(input_data=ts_test.data, incomp_data=incomp_data, recov_data=algo.recov_data, nbr_series=3, subplot=True, algorithm=algo.algorithm, cont_rate=str(cont_rate), display=False, save_path=save_dir_plot, verbose=False)

                            runs_plots_scores.setdefault(str(dataset_s), {}).setdefault(str(pattern), {}).setdefault(str(algorithm), {}).setdefault(str(optimizer_value), {})[str(x)] = {"scores": algo.metrics}

                        print(f"done!\n\n")
                #save_dir_runs = save_dir + "/_details/run_" + str(i_run) + "/" + dataset
                #if verbose:
                #    print("\nruns saved in : ", save_dir_runs)
                #self.generate_plots(runs_plots_scores=runs_plots_scores, ticks=x_axis, metrics=metrics, subplot=True, y_size=y_p_size, save_dir=save_dir_runs, display=False, verbose=verbose)
                #self.generate_plots(runs_plots_scores=runs_plots_scores, ticks=x_axis, metrics=metrics, subplot=False, y_size=y_p_size, save_dir=save_dir_runs, display=False, verbose=verbose)
                #self.generate_reports_txt(runs_plots_scores=runs_plots_scores, save_dir=save_dir_runs, dataset=dataset, metrics=metrics, run=i_run, verbose=verbose)
                #self.generate_reports_excel(runs_plots_scores, save_dir_runs, dataset, i_run, verbose=verbose)
                run_storage.append(runs_plots_scores)

        plt.close('all')  # Close all open figures

        for x, m in enumerate(reversed(metrics)):
            #tag = True if x == (len(metrics)-1) else False
            scores_list, algos, sets = self.avg_results(*run_storage, metric=m)
            _ = self.generate_heatmap(scores_list=scores_list, algos=algos, sets=sets, metric=m, save_dir=save_dir, display=False)

        run_averaged = self.average_runs_by_names(run_storage)

        benchmark_end = time.time()
        total_time_benchmark = round(benchmark_end - benchmark_time, 4)
        print(f"\n> logs: benchmark - Execution Time: {total_time_benchmark} seconds\n")

        verb = True

        for scores in run_averaged:
            all_keys = list(scores.keys())
            dataset_name = str(all_keys[0])
            save_dir_agg_set = save_dir + "/" + dataset_name

            self.generate_reports_txt(runs_plots_scores=scores, save_dir=save_dir_agg_set, dataset=dataset_name, metrics=metrics, rt=total_time_benchmark, run=-1)
            self.generate_plots(runs_plots_scores=scores, ticks=x_axis, metrics=metrics, subplot=True, y_size=y_p_size, save_dir=save_dir_agg_set, display=verb)

        print("\nThe results are saved in : ", save_dir, "\n")
        self.list_results = run_averaged
        self.aggregate_results = scores_list

# direct implementation of benchmark

In [ ]:
import os
import yaml
from imputegap.recovery.manager import TimeSeries
from imputegap.tools import utils
from imputegap.recovery.benchmark import Benchmark
import datetime
import time
import math
import matplotlib.pyplot as plt
bench = Benchmark()
dir = "../data/04_working_files/"
with open(dir + "file_list.yaml", 'r') as file:
    datafiles = yaml.safe_load(file)
datafiles = ["../" + d for d in datafiles]
with open(dir + "file_list_missing.yaml", 'r') as file:
    datafiles_missing = yaml.safe_load(file)
datafiles_missing = ["../" + d for d in datafiles_missing]
ts = TimeSeries()

bench = Benchmark()
algorithms=ts.algorithms
datasets=datafiles
ts_missing=datafiles_missing
patterns=["mcar"]
x_axis=[0.1]
optimizers=["ray_tune"]
metrics=["*"]
normalizer=""
nbr_series=11
nbr_vals=100000
dl_ratio=0.9
verbose=True
save_dir="./imputegap_assets/benchmark"
runs=1
x=0.2

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

run_storage = []
not_optimized = ["none"]
mean_group = ["mean", "MeanImpute", "min", "MinImpute", "zero", "ZeroImpute", "MeanImputeBySeries", "meanimpute", "minimpute", "zeroimpute", "meanimputebyseries"]

if not isinstance(algorithms, list):
    raise TypeError(f"'algorithms' must be a list, but got {type(algorithms).__name__}")
if not isinstance(datasets, list):
    raise TypeError(f"'datasets' must be a list, but got {type(datasets).__name__}")
if not isinstance(patterns, list):
    raise TypeError(f"'patterns' must be a list, but got {type(patterns).__name__}")
if not isinstance(x_axis, list):
    raise TypeError(f"'x_axis' must be a list, but got {type(x_axis).__name__}")

if "*" in metrics or "all" in metrics:
    metrics = utils.list_of_metrics()
if "*" in metrics or "all" in algorithms:
    all_algs = utils.list_of_algorithms()
    algorithms = [item for item in all_algs if item.upper() != "MPIN"]

directory_now = datetime.datetime.now()
directory_time = directory_now.strftime("%y_%m_%d_%H_%M_%S")
save_dir = save_dir + "/" + "bench_" + directory_time

benchmark_time = time.time()
for i_run in range(0, abs(runs)):
    for i, dataset in enumerate(datasets):
        runs_plots_scores = {}
        block_size_mcar = 10
        y_p_size = max(4, len(algorithms)*0.275)

        if verbose:
            print("\n1. evaluation launch for", dataset, "\n")
        ts_test = TimeSeries()
        default_data = TimeSeries()

        header = False
        if dataset == "eeg-reading" or dataset == "eegreading":
            header = True

        reshp = False
        default_data.load_series(data=utils.search_path(dataset), header=header, verbose=False)
        Mdef, Ndef = default_data.data.shape

        if Ndef > nbr_vals or Mdef > nbr_series:
            reshp = True
            print(f"\nThe dataset contains a large number of values {default_data.data.shape}, which may be too much for some algorithms to handle efficiently. Consider reducing the number of series or the volume of data.")
        default_data = None

        ts_test.load_series(data=utils.search_path(dataset), nbr_series=nbr_series, nbr_val=nbr_vals, header=header)
        _, N = ts_test.data.shape

        if reshp:
            print(f"Benchmarking module has reduced the shape to {ts_test.data.shape}.\n")

        if N < 250:
            print(f"The block size is too high for the number of values per series, reduce to 2\n")
            block_size_mcar = 2

        if normalizer in utils.list_of_normalizers():
            ts_test.normalize(verbose=verbose)

        for pattern in patterns:
            if verbose:
                print("\n2. contamination of", dataset, "with pattern", pattern, "\n")

            for algorithm in algorithms:
                has_been_optimized = False

                if verbose:
                    print("\n3. algorithm evaluated", algorithm, "with", pattern, "\n")
                else:
                    print(f"{algorithm} is tested with {pattern}, started at {time.strftime('%Y-%m-%d %H:%M:%S')}.")

                incomp_data = TimeSeries()
                incomp_data = incomp_data.load_series(data = ts_missing[i]).data

                for optimizer in optimizers:
                    algo = utils.config_impute_algorithm(incomp_data=incomp_data, algorithm=algorithm, verbose=verbose)

                    if isinstance(optimizer, dict):
                        optimizer_gt = {"input_data": ts_test.data, **optimizer}
                        optimizer_value = optimizer.get('optimizer')  # or optimizer['optimizer']

                        if not has_been_optimized and algorithm not in mean_group and algorithm not in not_optimized:
                            if verbose:
                                print("\n5. AutoML to set the parameters", optimizer, "\n")
                            i_opti = self._config_optimization(0.20, ts_test, pattern, algorithm, block_size_mcar)

                            if utils.check_family("DeepLearning", algorithm):
                                i_opti.impute(user_def=False, params=optimizer_gt, tr_ratio=0.80)
                            else:
                                i_opti.impute(user_def=False, params=optimizer_gt)

                            utils.save_optimization(optimal_params=i_opti.parameters, algorithm=algorithm, dataset=dataset, optimizer="e")

                            has_been_optimized = True
                        else:
                            if verbose:
                                print("\n5. AutoML already optimized...\n")

                        if algorithm not in mean_group and algorithm not in not_optimized:
                            if i_opti.parameters is None:
                                opti_params = utils.load_parameters(query="optimal", algorithm=algorithm, dataset=dataset, optimizer="e")
                                if verbose:
                                    print("\n6. imputation", algorithm, "with optimal parameters from files", *opti_params)
                            else:
                                opti_params = i_opti.parameters
                                if verbose:
                                    print("\n6. imputation", algorithm, "with optimal parameters from object", *opti_params)
                        else:
                            if verbose:
                                print("\n5. No AutoML launches without optimal params for", algorithm, "\n")
                            opti_params = None
                    else:
                        if verbose:
                            print("\n5. Default parameters have been set the parameters", optimizer, "for", algorithm, "\n")
                        optimizer_value = optimizer
                        opti_params = None

                    start_time_imputation = time.time()

                    if not bench._benchmark_exception(dataset, algorithm, pattern, 0.2):
                        if utils.check_family("DeepLearning", algorithm) or utils.check_family("LLMs", algorithm):
                            if 0.2 > round(1-dl_ratio, 2):
                                algo.recov_data = incomp_data
                            else:
                                algo.impute(params=opti_params, tr_ratio=dl_ratio)
                        else:
                            algo.impute(params=opti_params)
                    else:
                        algo.recov_data = incomp_data

                    end_time_imputation = time.time()

                    algo.score(input_data=ts_test.data, recov_data=algo.recov_data, verbose=False)

                    if "*" not in metrics and "all" not in metrics:
                        algo.metrics = {k: algo.metrics[k] for k in metrics if k in algo.metrics}

                    time_imputation = (end_time_imputation - start_time_imputation) * 1000
                    if time_imputation < 1:
                        time_imputation = 1
                    log_time_imputation = math.log10(time_imputation) if time_imputation > 0 else None

                    algo.metrics["RUNTIME"] = time_imputation
                    algo.metrics["RUNTIME_LOG"] = log_time_imputation

                    dataset_s = dataset
                    if "-" in dataset:
                        dataset_s = dataset.replace("-", "")

                    save_dir_plot = save_dir + "/" + dataset_s + "/" + pattern + "/recovery/"
                    cont_rate = int(0.2*100)
                    ts_test.plot(input_data=ts_test.data, incomp_data=incomp_data, recov_data=algo.recov_data, nbr_series=3, subplot=True, algorithm=algo.algorithm, cont_rate=str(cont_rate), display=False, save_path=save_dir_plot, verbose=False)

                    runs_plots_scores.setdefault(str(dataset_s), {}).setdefault(str(pattern), {}).setdefault(str(algorithm), {}).setdefault(str(optimizer_value), {})[str(x)] = {"scores": algo.metrics}

                print(f"done!\n\n")
        #save_dir_runs = save_dir + "/_details/run_" + str(i_run) + "/" + dataset
        #if verbose:
        #    print("\nruns saved in : ", save_dir_runs)
        #self.generate_plots(runs_plots_scores=runs_plots_scores, ticks=x_axis, metrics=metrics, subplot=True, y_size=y_p_size, save_dir=save_dir_runs, display=False, verbose=verbose)
        #self.generate_plots(runs_plots_scores=runs_plots_scores, ticks=x_axis, metrics=metrics, subplot=False, y_size=y_p_size, save_dir=save_dir_runs, display=False, verbose=verbose)
        #self.generate_reports_txt(runs_plots_scores=runs_plots_scores, save_dir=save_dir_runs, dataset=dataset, metrics=metrics, run=i_run, verbose=verbose)
        #self.generate_reports_excel(runs_plots_scores, save_dir_runs, dataset, i_run, verbose=verbose)
        run_storage.append(runs_plots_scores)

plt.close('all')  # Close all open figures

for x, m in enumerate(reversed(metrics)):
    #tag = True if x == (len(metrics)-1) else False
    scores_list, algos, sets = bench.avg_results(*run_storage, metric=m)
    _ = bench.generate_heatmap(scores_list=scores_list, algos=algos, sets=sets, metric=m, save_dir=save_dir, display=False)

run_averaged = bench.average_runs_by_names(run_storage)

benchmark_end = time.time()
total_time_benchmark = round(benchmark_end - benchmark_time, 4)
print(f"\n> logs: benchmark - Execution Time: {total_time_benchmark} seconds\n")

verb = True

for scores in run_averaged:
    all_keys = list(scores.keys())
    dataset_name = str(all_keys[0])
    save_dir_agg_set = save_dir + "/" + dataset_name

    bench.generate_reports_txt(runs_plots_scores=scores, save_dir=save_dir_agg_set, dataset=dataset_name, metrics=metrics, rt=total_time_benchmark, run=-1)
    bench.generate_plots(runs_plots_scores=scores, ticks=x_axis, metrics=metrics, subplot=True, y_size=y_p_size, save_dir=save_dir_agg_set, display=verb)

print("\nThe results are saved in : ", save_dir, "\n")
bench.list_results = run_averaged
bench.aggregate_results = scores_list

# using of imputegap
- testing all algorithms 
- sorting out non functional algorithms
  - ["GPT4TS", "MissNet","NuwaTS"] dont seem to work

In [18]:
ts = TimeSeries()
ts.optimizers

['bayesian', 'greedy', 'particle_swarm', 'ray_tune', 'successive_halving']

In [30]:
datafiles

['../data/04_working_files/ts_feature_Wind_speed.csv',
 '../data/04_working_files/ts_feature_Wind_direction.csv',
 '../data/04_working_files/ts_feature_Relative_humidity.csv',
 '../data/04_working_files/ts_feature_Air_temperature.csv',
 '../data/04_working_files/ts_feature_Air_pressure.csv']

In [1]:
from imputegap.recovery.benchmark import Benchmark
from imputegap.recovery.manager import TimeSeries
ts = TimeSeries()
algorithms=ts.algorithms

In [2]:

algorithms

['BRITS',
 'BayOTIDE',
 'BitGraph',
 'CDRec',
 'DeepMVI',
 'DynaMMo',
 'GAIN',
 'GPT4TS',
 'GRIN',
 'GROUSE',
 'HKMF_T',
 'IIM',
 'Interpolation',
 'IterativeSVD',
 'KNNImpute',
 'MICE',
 'MPIN',
 'MRNN',
 'MeanImpute',
 'MeanImputeBySeries',
 'MinImpute',
 'MissForest',
 'MissNet',
 'NuwaTS',
 'PRISTI',
 'ROSL',
 'SPIRIT',
 'STMVL',
 'SVT',
 'SoftImpute',
 'TKCM',
 'TRMF',
 'XGBOOST',
 'ZeroImpute']

In [3]:
bad = ["GPT4TS", "MissNet","NuwaTS"]
for bad_alg in bad:
    if bad_alg in algorithms:
        algorithms.remove(bad_alg)
algorithms

['BRITS',
 'BayOTIDE',
 'BitGraph',
 'CDRec',
 'DeepMVI',
 'DynaMMo',
 'GAIN',
 'GRIN',
 'GROUSE',
 'HKMF_T',
 'IIM',
 'Interpolation',
 'IterativeSVD',
 'KNNImpute',
 'MICE',
 'MPIN',
 'MRNN',
 'MeanImpute',
 'MeanImputeBySeries',
 'MinImpute',
 'MissForest',
 'PRISTI',
 'ROSL',
 'SPIRIT',
 'STMVL',
 'SVT',
 'SoftImpute',
 'TKCM',
 'TRMF',
 'XGBOOST',
 'ZeroImpute']

In [4]:
import os
import yaml
from imputegap.recovery.manager import TimeSeries
from imputegap.tools import utils
from imputegap.recovery.benchmark import Benchmark
import datetime
import time
import math
import matplotlib.pyplot as plt
bench = Benchmark()
dir = "../data/04_working_files/"
with open(dir + "file_list.yaml", 'r') as file:
    datafiles = yaml.safe_load(file)
datafiles = ["../" + d for d in datafiles]
with open(dir + "file_list_missing.yaml", 'r') as file:
    datafiles_missing = yaml.safe_load(file)
datafiles_missing = ["../" + d for d in datafiles_missing]

In [12]:
[datafiles[2]]

['../data/04_working_files/ts_feature_Relative_humidity.csv']

In [16]:
from imputegap.recovery.benchmark import Benchmark
from imputegap.recovery.manager import TimeSeries
ts = TimeSeries()


my_opt = ["default_params"]

my_datasets = ["eeg-alcohol"]

my_patterns = ["mcar"]

range = [0.05, 0.1, 0.2, 0.4, 0.6, 0.8]

my_metrics = ["*"]
#optimizers="ray_tune",
# launch the evaluation
bench = Benchmark()
bench.eval(algorithms=algorithms, datasets=[datafiles[2]], patterns=my_patterns, x_axis=range, metrics=my_metrics,  verbose=True,nbr_vals=1000)


1. evaluation launch for ../data/04_working_files/ts_feature_Relative_humidity.csv 


The dataset contains a large number of values (10, 87648), which may be too much for some algorithms to handle efficiently. Consider reducing the number of series or the volume of data.

(SYS) The dataset is loaded from ../data/04_working_files/ts_feature_Relative_humidity.csv

Benchmarking module has reduced the shape to (10, 1000).

> logs: normalization (z_score) of the data - runtime: 0.0003 seconds

2. contamination of ../data/04_working_files/ts_feature_Relative_humidity.csv with pattern mcar 


3. algorithm evaluated BRITS with mcar 


4. missing values (series&values) set to 0.05 for x_axis


(CONT) missigness pattern: MCAR
	selected series: 9
	percentage of contaminated series: 5.0%
	rate of missing data per series: 5.0%
	block size: 10
	security offset: [0-100]
	seed value: 42

5. Default parameters have been set the parameters default_params for BRITS 


(SYS) Inner files loaded :  /home/t

  0%|          | 0/1000 [00:00<?, ?it/s]

T_id = 0, train_rmse = 9.174, test_rmse= 100008.348


 10%|█         | 102/1000 [00:04<01:08, 13.16it/s]

T_id = 100, train_rmse = 5.334, test_rmse= 100007.605


...

100%|██████████| 1/1 [00:00<00:00,  4.33it/s, avg_epoch_loss=253, epoch=24]




reconstruction


100%|██████████| 1/1 [00:01<00:00,  1.11s/it]



reconstruction.shape = (10, 1000)

> logs: imputation priSTI - Execution Time: 8.6589 seconds


4. missing values (series&values) set to 0.2 for x_axis


(CONT) missigness pattern: MCAR
	selected series: 2, 9
	percentage of contaminated series: 20.0%
	rate of missing data per series: 20.0%
	block size: 10
	security offset: [0-100]
	seed value: 42

5. Default parameters have been set the parameters default_params for PRISTI 

(EVAL) pristi ended with errors, the imputed time series contains NaN values.
	Please, check your configuration, the algorithm might not work with the percentage of contamination or the pattern chosen.

4. missing values (series&values) set to 0.4 for x_axis


(CONT) missigness pattern: MCAR
	selected series: 1, 2, 6, 9
	percentage of contaminated series: 40.0%
	rate of missing data per series: 40.0%
	block size: 10
	security offset: [0-100]
	seed value: 42

5. Default parameters have been set the parameters default_params for PRISTI 

(EVAL) pristi ended with err

FileNotFoundError: [Errno 2] No such file or directory: './imputegap_assets/benchmark/bench_25_10_20_00_15_47/../data/04_working_files/ts_feature_Relative_humidity.csv/mcar/error/report_mcar_../data/04_working_files/ts_feature_Relative_humidity.csv.txt'

In [6]:
%tb

SystemExit: 2